In [15]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/delhi-climate-data/DailyDelhiClimateTest.csv


In [16]:
## loading data into pandas dataframe
df = pd.read_csv('/kaggle/input/delhi-climate-data/DailyDelhiClimateTest.csv')

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114 entries, 0 to 113
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   date          114 non-null    object 
 1   meantemp      114 non-null    float64
 2   humidity      114 non-null    float64
 3   wind_speed    114 non-null    float64
 4   meanpressure  114 non-null    float64
dtypes: float64(4), object(1)
memory usage: 4.6+ KB


In [18]:
df

,date,meantemp,humidity,wind_speed,meanpressure
0,2017-01-01,15.913043,85.869565,2.743478,59.000000
1,2017-01-02,18.500000,77.222222,2.894444,1018.277778
2,2017-01-03,17.111111,81.888889,4.016667,1018.333333
3,2017-01-04,18.700000,70.050000,4.545000,1015.700000
4,2017-01-05,18.388889,74.944444,3.300000,1014.333333
...,...,...,...,...,...
109,2017-04-20,34.500000,27.500000,5.562500,998.625000
110,2017-04-21,34.250000,39.375000,6.962500,999.875000
111,2017-04-22,32.900000,40.900000,8.890000,1001.600000
112,2017-04-23,32.875000,27.500000,9.962500,1002.125000


# Q1.1
### Built the complete model manually on a paper. Compute the least square estimators for the same using Python. Report the magnitude of the affect of “mean temperature”, “wind speed” and “mean pressure” on the humidity in Delhi. Further add their interpretations.

In [19]:
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler

X = df[['meantemp', 'wind_speed', 'meanpressure']] # Features
y = df['humidity']  # target
X = np.column_stack((np.ones(len(X)), X))

coeff = np.linalg.inv(X.T @ X) @ X.T @ y
print(f"shape: {coeff.shape}")

## b0 -> intercept
## b1,b2,b3 = coefficients of meantemp, windspeed and meanpressure resp.
coeff_names = ['b0', 'b1', 'b2', 'b3']  
for name, value in zip(coeff_names, coeff):
    print(f"{name}: {value}")

shape: (4,)
b0: 127.71884020283386
b1: -2.465802058648585
b2: -0.8214944440240082
b3: -0.011185002811553504


#### The affects of of “mean temperature”, “wind speed” and “mean pressure” on the humidity will be the magnitude of their coefficients i.e. b1, b2 and b3 respectively.
#### Interpretation of the coefficients:
#### The coefficient estimates represent the change in the dependent variable (humidity) for a one-unit increase in the corresponding independent variable(meantemp, windspeed or meanpressure), holding other variables constant.
#### The magnitude of the coefficient indicates the strength and direction of the relationship between each independent variable and the dependent variable.


# Q1.2
### “It is believed that humidity increases significantly after 21.March.2017”. Verify this statement by making suitable changes in your model. Write the new model manually and justify the results for the same by using Python.

In [20]:
import pandas as pd
import numpy as np

df = pd.read_csv('/kaggle/input/delhi-climate-data/DailyDelhiClimateTest.csv')

df['after_march21'] = (df['date'] >= '2017-03-21').astype(int)  
X = df[['meantemp', 'wind_speed', 'meanpressure', 'after_march21']]
X = np.column_stack((np.ones(len(X)), X))  

y = df['humidity'].values

coeff = np.linalg.inv(X.T @ X) @ X.T @ y
print(f"shape: {coeff.shape}")
coeff_names = ['b0', 'b1', 'b2', 'b3', 'b4']
for name, value in zip(coeff_names, coeff):
    print(f"{name}: {value}")
    
# print(X.shape)

shape: (5,)
b0: 121.99082582595463
b1: -2.114436391890142
b2: -0.7703032592573688
b3: -0.011790057264331862
b4: -5.57172490223461


#### In the model, I made the following changes:
#### Added after_march21 column in X such that if date in original data is greater than '2017-03-21' then set the value as 1 otherwise 0 and then applied OLS and obtained the coefficients.
#### Thing to note is that the coefficient of after_march21 i.e. b4 is significant and its value is negative suggesting that after 21 march humidity decreases significantly. Thus, the statement mentioned in the question is wrong.

# Q.1.3
### Your goal is to estimate the unbiased efficient betas. Thus, check for the violation of OLS assumptions in your latest model including homoscedasticity and no autocorrelation. Provide the final results manually.

In [21]:
## Applying white's test for heteroskedasticity

import statsmodels.api as sm
from scipy.stats import chi2

df = pd.read_csv('/kaggle/input/delhi-climate-data/DailyDelhiClimateTest.csv')
X = df[['meantemp', 'wind_speed', 'meanpressure']]
y = df['humidity']
X = sm.add_constant(X)  

model = sm.OLS(y, X).fit()
residuals = model.resid
residuals_squared = residuals**2


X_white = df[['meantemp', 'wind_speed', 'meanpressure']]
X_white = np.column_stack((np.ones(len(X_white)), X_white))  
model_white = sm.OLS(residuals_squared, X_white).fit()

R_squared_white = model_white.rsquared
n = len(residuals)
White_statistic = n * R_squared_white

df_model_white = len(X_white[0]) - 1
White_p_value = 1 - chi2.cdf(White_statistic, df_model_white)

print(f"White Test Statistic: {White_statistic}")
print(f"P-value: {White_p_value}")

alpha = 0.05

if White_p_value < alpha:
    print("Heteroscedasticity exists.")
else:
    print("No heteroscedasticity exists.")



White Test Statistic: 10.411007411706194
P-value: 0.015376897903841114
Heteroscedasticity exists.


#### According to white's test if the chisquare value obtained above is greater than the critical value at the chosen level of significance, we conclude there is heteroskedasticity.

In [22]:
## Breusch Godfrey for detecting autocorrelation

from statsmodels.stats.diagnostic import acorr_breusch_godfrey
import statsmodels.api as sm

df = pd.read_csv('/kaggle/input/delhi-climate-data/DailyDelhiClimateTest.csv')
X = df[['meantemp', 'wind_speed', 'meanpressure']]
y = df['humidity']
X = sm.add_constant(X)  

model = sm.OLS(y, X).fit()

bg_test_results = acorr_breusch_godfrey(model, nlags=1)

bg_test_statistic = bg_test_results[0]
bg_pvalue = bg_test_results[1]

## alpha = 0.05
if bg_pvalue > 0.05:
    print("No evidence of autocorrelation (Breusch-Godfrey test p-value:", bg_pvalue, ")")
else:
    print("Autocorrelation detected (Breusch-Godfrey test p-value:", bg_pvalue, ")")


Autocorrelation detected (Breusch-Godfrey test p-value: 6.78173757913976e-14 )


#### If the obtained test statistic (n-p)*R^2 is greater than the critical value at the chosen level of significance, we reject the null hypothesis of no autocorrelation.

## Therefore in our data, both heteroskedasticity and autocorrelation is present.

# Q1.4
### Use suitable remedies for any violation of the assumptions and highlight the unbiased and efficient estimates for the model. Provide the final betas in the given space.

In [23]:
## dealing with heteroskedasticity using weighted least squares

import statsmodels.api as sm

df = pd.read_csv('/kaggle/input/delhi-climate-data/DailyDelhiClimateTest.csv')
X = df[['meantemp', 'wind_speed', 'meanpressure']]
y = df['humidity']
X = sm.add_constant(X)  

model = sm.OLS(y, X).fit()

y_pred = model.predict(X)
residuals = y - y_pred

weights = 1 / residuals**2
wls_model = sm.WLS(y, X, weights=weights)
wls_results = wls_model.fit()

final_betas = wls_results.params

print(final_betas)


const           127.426134
meantemp         -2.417290
wind_speed       -0.873859
meanpressure     -0.011758
dtype: float64


#### In the above code, I provided the remedy for heteroskedasticity in which i used weighted least squares taking weight = 1/residual^2

In [24]:
## To cure autocorrelation, I am applying first difference transformation
import statsmodels.api as sm

df = pd.read_csv('/kaggle/input/delhi-climate-data/DailyDelhiClimateTest.csv')
X = df[['meantemp', 'wind_speed', 'meanpressure']]
y = df['humidity']


y_diff = np.diff(y)
X_diff = np.diff(X, axis=0)  
X_diff = sm.add_constant(X_diff)

ols_model = sm.OLS(y_diff, X_diff)
ols_results = ols_model.fit()

## checking with durbin watson test if autocorrelation removed with this transformation.
residuals = ols_results.resid
db = sm.stats.stattools.durbin_watson(residuals)
print(f"the value of db statistic: {db}")

final_betas = ols_results.params

print(final_betas)


the value of db statistic: 2.003212106412154
[-0.21520153 -1.73433686 -0.2526155  -0.00437045]


#### As observed above the value of db statistic comes out to be near to 2, thus, autocorrelation removed with the first order transformation.

# Q1.5
### If there is no autocorrelation, still provide the results for the first difference analysis and GLS in the space below.

In [25]:
## Applying GLS combined with first difference transformation:

import statsmodels.api as sm

df = pd.read_csv('/kaggle/input/delhi-climate-data/DailyDelhiClimateTest.csv')
X = df[['meantemp', 'wind_speed', 'meanpressure']]
y = df['humidity']

y_diff = np.diff(y)
X_diff = np.diff(X, axis=0)  

X_diff_const = sm.add_constant(X_diff)
ols_model = sm.OLS(y_diff, X_diff_const)
ols_results = ols_model.fit()
residuals = ols_results.resid

weights = 1 / residuals**2
wls_model = sm.WLS(y_diff, X_diff_const, weights=weights)
wls_results = wls_model.fit()

residuals = ols_results.resid
db = sm.stats.stattools.durbin_watson(residuals)
print(f"the value of db statistic: {db}")

final_betas = wls_results.params

print(final_betas)



the value of db statistic: 2.003212106412154
[-0.30052404 -1.73800941 -0.27219621 -0.00397047]


#### In the above code cell, I combined the two methodologies used in 1.4 to remove heteroskedasticity and autocorrelation at the same time. Heteroskedasticity is cured by weighted least squares and autocorrelation is cured by first difference transformation which I applied because rho was unknown.